# Import Contour Data

In [1]:
import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
ACS=Dataset('META3.1exp_DT_allsat_Anticyclonic_short_19930101_20200307.nc')

In [3]:
ACL=Dataset('META3.1exp_DT_allsat_Anticyclonic_long_19930101_20200307.nc')

In [4]:
print(ACS.variables.keys())

dict_keys(['amplitude', 'cost_association', 'effective_area', 'effective_contour_height', 'effective_contour_latitude', 'effective_contour_longitude', 'effective_contour_shape_error', 'effective_radius', 'inner_contour_height', 'latitude', 'latitude_max', 'longitude', 'longitude_max', 'num_contours', 'num_point_e', 'num_point_s', 'observation_flag', 'observation_number', 'speed_area', 'speed_average', 'speed_contour_height', 'speed_contour_latitude', 'speed_contour_longitude', 'speed_contour_shape_error', 'speed_radius', 'time', 'track', 'uavg_profile'])


# Import ADT Data

In [5]:
SLA=Dataset('../copernicus/cmems_obs-sl_glo_phy-ssh_my_allsat-l4-duacs-0.25deg_P1D_1704119807053.nc')

In [6]:
print(SLA.variables.keys())

dict_keys(['adt', 'crs', 'latitude', 'time', 'longitude'])


In [7]:
adt=SLA.variables['adt'][:]

In [8]:
adt.shape

(3653, 81, 181)

In [9]:
maplat=SLA.variables['latitude'][:]
maplon=SLA.variables['longitude'][:]

In [10]:
maplat

masked_array(data=[24.875, 25.125, 25.375, 25.625, 25.875, 26.125, 26.375,
                   26.625, 26.875, 27.125, 27.375, 27.625, 27.875, 28.125,
                   28.375, 28.625, 28.875, 29.125, 29.375, 29.625, 29.875,
                   30.125, 30.375, 30.625, 30.875, 31.125, 31.375, 31.625,
                   31.875, 32.125, 32.375, 32.625, 32.875, 33.125, 33.375,
                   33.625, 33.875, 34.125, 34.375, 34.625, 34.875, 35.125,
                   35.375, 35.625, 35.875, 36.125, 36.375, 36.625, 36.875,
                   37.125, 37.375, 37.625, 37.875, 38.125, 38.375, 38.625,
                   38.875, 39.125, 39.375, 39.625, 39.875, 40.125, 40.375,
                   40.625, 40.875, 41.125, 41.375, 41.625, 41.875, 42.125,
                   42.375, 42.625, 42.875, 43.125, 43.375, 43.625, 43.875,
                   44.125, 44.375, 44.625, 44.875],
             mask=False,
       fill_value=1e+20,
            dtype=float32)

In [11]:
maplon

masked_array(data=[134.875, 135.125, 135.375, 135.625, 135.875, 136.125,
                   136.375, 136.625, 136.875, 137.125, 137.375, 137.625,
                   137.875, 138.125, 138.375, 138.625, 138.875, 139.125,
                   139.375, 139.625, 139.875, 140.125, 140.375, 140.625,
                   140.875, 141.125, 141.375, 141.625, 141.875, 142.125,
                   142.375, 142.625, 142.875, 143.125, 143.375, 143.625,
                   143.875, 144.125, 144.375, 144.625, 144.875, 145.125,
                   145.375, 145.625, 145.875, 146.125, 146.375, 146.625,
                   146.875, 147.125, 147.375, 147.625, 147.875, 148.125,
                   148.375, 148.625, 148.875, 149.125, 149.375, 149.625,
                   149.875, 150.125, 150.375, 150.625, 150.875, 151.125,
                   151.375, 151.625, 151.875, 152.125, 152.375, 152.625,
                   152.875, 153.125, 153.375, 153.625, 153.875, 154.125,
                   154.375, 154.625, 154.875, 155.1

# Calculate Time Range

In [12]:
time=ACS.variables['time'][:]

In [13]:
import datetime

In [26]:
datetime.datetime.strptime('1950-01-01','%Y-%m-%d')+datetime.timedelta(days=21915)

datetime.datetime(2010, 1, 1, 0, 0)

In [23]:
datetime.datetime.strptime('2009-01-01','%Y-%m-%d')-datetime.datetime.strptime('1950-01-01','%Y-%m-%d')

datetime.timedelta(days=21550)

Time from 2010-1-1 to 2020-1-1 should be 21915 to 25567.

# Data Transform

## Get Contour Coordinate within Northwest Pacific Area

In [16]:
time_acs=ACS.variables['time'][:]
center_lon_acs=ACS.variables['longitude'][:]
center_lat_acs=ACS.variables['latitude'][:]

In [17]:
time_acl=ACL.variables['time'][:]
center_lon_acl=ACL.variables['longitude'][:]
center_lat_acl=ACL.variables['latitude'][:]

In [18]:
import time as tm

保存为：

contour_coordinate:顶点的坐标，格式为[[50个lon],[50个lat]],即list中每个元素shape均为(2,50)

contour_time:顶点坐标获取的时间

contour_index:顶点坐标在原始数据中对应的序号

## Transform Coordinate into Grid

In [19]:
contour_coordinate_acs=np.load('contour_coordinate_acs.npy')
contour_time_acs=np.load('contour_time_acs.npy')
contour_index_acs=np.load('contour_index_acs.npy')

In [20]:
contour_coordinate_acl=np.load('contour_coordinate_acl.npy')
contour_time_acl=np.load('contour_time_acl.npy')
contour_index_acl=np.load('contour_index_acl.npy')

In [21]:
def point_in_polygon(x, y, poly_x, poly_y):    #判断grid坐标是否位于涡旋边界内部，输入grid lon, grid lat, contour lon, contour lat
    num_vertices = len(poly_x)
    inside = False
    j = num_vertices - 1
    for i in range(num_vertices):
        if ((poly_y[i] > y) != (poly_y[j] > y)) and (x < (poly_x[j] - poly_x[i]) * (y - poly_y[i]) / (poly_y[j] - poly_y[i]) + poly_x[i]):
            inside = not inside
        j = i
    return inside

改了个时间

In [28]:
grid_total=[]
start_time=tm.time()
for time in range(21550,21915):
    grid_data = np.zeros((len(maplat), len(maplon)))
    for k in range(len(contour_time_acs)):
        if contour_time_acs[k]==time:
            #print(k,time)
            for i in range(len(maplat)):
                for j in range(len(maplon)):
                    if point_in_polygon(maplon[j], maplat[i], contour_coordinate_acs[k][0], contour_coordinate_acs[k][1]):
                        grid_data[i, j] = 1
    
    grid_total.append(grid_data)
    end_time=tm.time()
    print(str(time)+', '+str(end_time-start_time)+'s')

21550, 5.028570890426636s
21551, 8.362133264541626s
21552, 11.695943355560303s
21553, 15.023573875427246s
21554, 18.3554527759552s
21555, 23.352904558181763s
21556, 27.541574478149414s
21557, 31.719550609588623s
21558, 35.906469106674194s
21559, 38.425766944885254s
21560, 41.79347062110901s
21561, 45.128305435180664s
21562, 50.94915437698364s
21563, 59.229594707489014s
21564, 65.89913129806519s
21565, 73.38493704795837s
21566, 82.52922558784485s
21567, 91.64805626869202s
21568, 97.47659373283386s
21569, 101.65029573440552s
21570, 105.003000497818s
21571, 107.5321888923645s
21572, 112.54880023002625s
21573, 116.73289370536804s
21574, 120.08746671676636s
21575, 122.62249565124512s
21576, 126.8323175907135s
21577, 130.18715381622314s
21578, 134.38955354690552s
21579, 141.05343508720398s
21580, 147.72330331802368s
21581, 155.20223474502563s
21582, 161.02506136894226s
21583, 169.34944224357605s
21584, 179.3330626487732s
21585, 185.1517255306244s
21586, 192.63491559028625s
21587, 200.9618103

21863, 1187.4117827415466s
21864, 1194.8442223072052s
21865, 1201.457858324051s
21866, 1208.8764152526855s
21867, 1213.8442759513855s
21868, 1218.8118214607239s
21869, 1223.7788310050964s
21870, 1226.2820363044739s
21871, 1228.8013770580292s
21872, 1230.4920382499695s
21873, 1232.9989535808563s
21874, 1235.5055303573608s
21875, 1239.6523578166962s
21876, 1243.8148441314697s
21877, 1245.5006053447723s
21878, 1248.010798215866s
21879, 1249.703771829605s
21880, 1252.2146973609924s
21881, 1254.729153394699s
21882, 1255.6036417484283s
21883, 1256.4750742912292s
21884, 1257.3445262908936s
21885, 1259.8522152900696s
21886, 1262.3625707626343s
21887, 1265.6857171058655s
21888, 1269.0188536643982s
21889, 1272.3470222949982s
21890, 1275.6831448078156s
21891, 1278.1961600780487s
21892, 1283.1610074043274s
21893, 1288.1381976604462s
21894, 1291.4713468551636s
21895, 1293.9872560501099s
21896, 1296.5082218647003s
21897, 1299.8344979286194s
21898, 1302.3529324531555s
21899, 1304.0522921085358s
21900

In [29]:
grid_total=np.array(grid_total)
np.save('grid_acs_test',grid_total)

In [30]:
grid_total=[]
start_time=tm.time()
for time in range(21550,21915):
    grid_data = np.zeros((len(maplat), len(maplon)))
    for k in range(len(contour_time_acl)):
        if contour_time_acl[k]==time:
            #print(k,time)
            for i in range(len(maplat)):
                for j in range(len(maplon)):
                    if point_in_polygon(maplon[j], maplat[i], contour_coordinate_acl[k][0], contour_coordinate_acl[k][1]):
                        grid_data[i, j] = 1
    
    grid_total.append(grid_data)
    end_time=tm.time()
    print(str(time)+', '+str(end_time-start_time)+'s')

21550, 63.423827171325684s
21551, 127.58807635307312s
21552, 191.69667291641235s
21553, 257.4975206851959s
21554, 320.9217793941498s
21555, 383.44980120658875s
21556, 444.4290282726288s
21557, 507.0369486808777s
21558, 572.042905330658s
21559, 637.8379147052765s
21560, 701.1305820941925s
21561, 763.7030906677246s
21562, 826.3735809326172s
21563, 890.6903185844421s
21564, 955.715788602829s
21565, 1023.2496597766876s
21566, 1089.9126002788544s
21567, 1157.4575848579407s
21568, 1222.5772211551666s
21569, 1288.751928806305s
21570, 1356.3644416332245s
21571, 1426.4546880722046s
21572, 1497.4039113521576s
21573, 1569.900603055954s
21574, 1642.3996510505676s
21575, 1714.9094715118408s
21576, 1786.56605219841s
21577, 1858.9989347457886s
21578, 1929.2641546726227s
21579, 2000.3099324703217s
21580, 2072.253182888031s
21581, 2145.7432811260223s
21582, 2218.3134202957153s
21583, 2291.7908256053925s
21584, 2366.8960359096527s
21585, 2442.6143910884857s
21586, 2518.2965970039368s
21587, 2592.4020180

21860, 20885.15242290497s
21861, 20946.756314516068s
21862, 21004.9433863163s
21863, 21064.042886972427s
21864, 21124.022850751877s
21865, 21184.750694274902s
21866, 21245.46265554428s
21867, 21305.427213191986s
21868, 21365.386599302292s
21869, 21423.743814706802s
21870, 21480.535836458206s
21871, 21537.192520856857s
21872, 21596.400146245956s
21873, 21656.491277694702s
21874, 21716.476206302643s
21875, 21782.11535000801s
21876, 21847.795590639114s
21877, 21912.683938980103s
21878, 21978.286883592606s
21879, 22045.634590625763s
21880, 22111.434213638306s
21881, 22177.839614629745s
21882, 22245.05967783928s
21883, 22314.0196557045s
21884, 22384.61812543869s
21885, 22456.024502515793s
21886, 22525.861679792404s
21887, 22596.602099895477s
21888, 22664.768227815628s
21889, 22732.86683821678s
21890, 22801.902944803238s
21891, 22870.11996293068s
21892, 22939.86165213585s
21893, 23008.815416812897s
21894, 23076.08517599106s
21895, 23143.508318424225s
21896, 23210.74475169182s
21897, 23277.33

In [31]:
grid_total=np.array(grid_total)
np.save('grid_acl_test',grid_total)